In [1]:
import pandas as pd
import numpy as np
import holidays
import calendar
from datetime import date
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [2]:
# --- 1. Load the original hourly data ---
DATA_PATH = "merged.csv"
TIME_COLUMN = 'measured_at'
GROUP_COLUMN = 'group_id'
TARGET_COLUMN = 'consumption'

print(f"Loading full hourly dataset from {DATA_PATH}...")
df = pd.read_csv(DATA_PATH)
df[TIME_COLUMN] = pd.to_datetime(df[TIME_COLUMN])

# --- 2. Define our aggregations ---
# We'll sum consumption, average the rest
weather_cols = ['avg_temp', 'avg_hum', 'wind', 'rain', 'air_pressure']
price_cols = ['eur_per_mwh']
static_cols = ['m_area', 'region', 'municipality', 'segment', 'p_type', 'c_bucket']

# Create a dictionary of aggregations
aggs = {}
aggs[TARGET_COLUMN] = 'sum'
for col in price_cols + weather_cols:
    aggs[col] = 'mean'
for col in static_cols:
    aggs[col] = 'first' # Static features

# --- 3. Set index and resample ---
print("Aggregating hourly data to monthly...")
# Set time as index for resampling
df = df.set_index(TIME_COLUMN)

# Group by group_id, then resample by Month Start ('MS')
monthly_df = df.groupby(GROUP_COLUMN).resample('MS').agg(aggs)

# --- 4. Clean up the new DataFrame ---
# Resampling creates a MultiIndex (group_id, measured_at). Reset it.
monthly_df = monthly_df.reset_index()

# Drop rows where we have no data (e.g., a group with no consumption in a month)
monthly_df = monthly_df.dropna(subset=[TARGET_COLUMN])

print(f"Aggregation complete. New monthly dataset has {len(monthly_df)} rows.")
monthly_df.info()

Loading full hourly dataset from merged.csv...
Aggregating hourly data to monthly...
Aggregation complete. New monthly dataset has 5040 rows.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5040 entries, 0 to 5039
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   group_id      5040 non-null   int64              
 1   measured_at   5040 non-null   datetime64[ns, UTC]
 2   consumption   5040 non-null   float64            
 3   eur_per_mwh   5040 non-null   float64            
 4   avg_temp      5040 non-null   float64            
 5   avg_hum       5040 non-null   float64            
 6   wind          5040 non-null   float64            
 7   rain          5040 non-null   float64            
 8   air_pressure  5040 non-null   float64            
 9   m_area        5040 non-null   object             
 10  region        5040 non-null   object             
 11  municipality  5040 non-null   o

In [3]:
print("--- Starting Monthly Feature Engineering (Direct Strategy) ---")
data = monthly_df.copy()

# Sort for correct lag creation
data = data.sort_values(by=[GROUP_COLUMN, TIME_COLUMN])

# --- 1. Create PREDICTOR Features (Our 'X' variables) ---

# A. Time Features
print("Creating time features...")
data['month'] = data[TIME_COLUMN].dt.month
data['year'] = data[TIME_COLUMN].dt.year
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12.0)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12.0)

fin_holidays = holidays.Finland()
def count_holidays_in_month(year, month):
    if not (isinstance(year, int) and isinstance(month, int)): return 0
    try:
        _, num_days = calendar.monthrange(year, month)
        count = 0
        for day in range(1, num_days + 1):
            if date(year, month, day) in fin_holidays:
                count += 1
        return count
    except ValueError:
        return 0
vectorized_holiday_count = np.vectorize(count_holidays_in_month)
data['num_holidays'] = vectorized_holiday_count(data['year'], data['month'])
data = data.drop(columns=['month']) # Drop the original 'month'

# B. Lag & Rolling Features (based on known 'consumption')
print("Creating lag and rolling features...")
df_grouped = data.groupby(GROUP_COLUMN)
lags = [1, 2, 3, 12]
windows = [12]
for lag in lags:
    data[f'consumption_lag_{lag}m'] = df_grouped['consumption'].shift(lag)
for window in windows:
    rolling_feat = df_grouped['consumption'].rolling(window=window, min_periods=1).mean()
    data[f'consumption_roll_mean_{window}m'] = rolling_feat.reset_index(level=0, drop=True)

# C. Exogenous Lag Features (based on known history)
exog_cols = ['eur_per_mwh', 'avg_temp', 'avg_hum', 'wind', 'rain', 'air_pressure']
for col in exog_cols:
    if col in data.columns:
        data[f'{col}_lag_12m'] = df_grouped[col].shift(12)

# --- 2. Create TARGET Features (Our 'y' variables) ---
# This is the new, critical part
print("Creating 12 future-month target columns...")
HORIZON = 12
target_cols = []
for i in range(1, HORIZON + 1):
    target_name = f'target_{i}m_ahead'
    # Shift backwards. 'target_1m_ahead' for Jan is the value from Feb.
    data[target_name] = df_grouped['consumption'].shift(-i)
    target_cols.append(target_name)

# --- 3. Final Cleanup ---
# Set categorical types
for col in static_cols:
    data[col] = data[col].astype('category')

data.info()

--- Starting Monthly Feature Engineering (Direct Strategy) ---
Creating time features...
Creating lag and rolling features...
Creating 12 future-month target columns...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5040 entries, 0 to 5039
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   group_id                   5040 non-null   int64              
 1   measured_at                5040 non-null   datetime64[ns, UTC]
 2   consumption                5040 non-null   float64            
 3   eur_per_mwh                5040 non-null   float64            
 4   avg_temp                   5040 non-null   float64            
 5   avg_hum                    5040 non-null   float64            
 6   wind                       5040 non-null   float64            
 7   rain                       5040 non-null   float64            
 8   air_pressure               5040 non-nul

In [8]:
# --- 1. Define Predictor Feature List ---
# These are the 'X' variables we will use for ALL 12 models
static_features = [col for col in data.columns if col in static_cols]
dynamic_features = [
    'year', 'month_sin', 'month_cos', 'num_holidays',
    'consumption_lag_1m', 'consumption_lag_2m', 'consumption_lag_3m', 'consumption_lag_12m',
    'consumption_roll_mean_12m',
    'eur_per_mwh_lag_12m',
    'avg_temp_lag_12m', 'avg_hum_lag_12m', 'wind_lag_12m', 
    'rain_lag_12m', 'air_pressure_lag_12m'
]
features = dynamic_features + static_features
print(f"--- Training with {len(features)} predictor features ---")

train_val_data = data.dropna(subset=features + target_cols)

# --- 2. Create Train/Validation Split ---
# This is the same simple 12-month split as before
val_cutoff = data[TIME_COLUMN].max() - pd.DateOffset(months=12)
train = data[data[TIME_COLUMN] <= val_cutoff].copy()
val = data[data[TIME_COLUMN] > val_cutoff].copy()

print(f"Training on {train[TIME_COLUMN].min()} to {train[TIME_COLUMN].max()}")
print(f"Validating on {val[TIME_COLUMN].min()} to {val[TIME_COLUMN].max()}")

# Create X data (this is the same for all models)
X_train = train[features]
X_val = val[features]

# --- 3. Define the Optuna Objective Function ---
# We will tune on a single, representative target: 6 months ahead
TARGET_TO_TUNE = 'target_6m_ahead' 

y_train_tune = train[TARGET_TO_TUNE]
y_val_tune = val[TARGET_TO_TUNE]

# Create the datasets just for tuning
tune_train_data = lgb.Dataset(X_train, label=y_train_tune, categorical_feature=static_features)
tune_val_data = lgb.Dataset(X_val, label=y_val_tune, reference=tune_train_data)

def objective(trial):
    params = {
        'objective': 'regression_l1', 'metric': 'rmse',
        'seed': 42, 'n_jobs': -1, 'verbose': -1,
        'n_estimators': 2000, 'feature_pre_filter': False,
        
        # Search Space (focused on regularization)
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 150),
    }

    model_tune = lgb.train(
        params,
        tune_train_data,
        valid_sets=[tune_val_data],
        valid_names=['validation'],
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )
    
    return model_tune.best_score['validation']['rmse']

# --- 4. Run the Optuna Study ---
print(f"--- Starting Optuna Search (Tuning for {TARGET_TO_TUNE}) ---")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

print("--- Search Complete ---")
print(f"Best Validation RMSE (for {TARGET_TO_TUNE}): {study.best_value:.2f}")
print("Best Parameters Found:")
print(study.best_params)

[I 2025-11-16 01:02:17,538] A new study created in memory with name: no-name-38e4a1e3-caea-423b-87d1-32c6539576bc


--- Training with 21 predictor features ---
Training on 2021-01-01 00:00:00+00:00 to 2023-09-01 00:00:00+00:00
Validating on 2023-10-01 00:00:00+00:00 to 2024-09-01 00:00:00+00:00
--- Starting Optuna Search (Tuning for target_6m_ahead) ---


[I 2025-11-16 01:02:17,865] Trial 0 finished with value: 480.3473500256947 and parameters: {'learning_rate': 0.026057394546719126, 'num_leaves': 34, 'subsample': 0.9381347664097133, 'colsample_bytree': 0.8873195301077801, 'lambda_l1': 5.088808338369863, 'lambda_l2': 6.640448146389386, 'min_child_samples': 63}. Best is trial 0 with value: 480.3473500256947.
[I 2025-11-16 01:02:18,253] Trial 1 finished with value: 485.41404701867293 and parameters: {'learning_rate': 0.012383086185110678, 'num_leaves': 24, 'subsample': 0.8412292775390067, 'colsample_bytree': 0.8278669217326391, 'lambda_l1': 0.980014751372782, 'lambda_l2': 1.945809695671526, 'min_child_samples': 38}. Best is trial 0 with value: 480.3473500256947.
[I 2025-11-16 01:02:18,472] Trial 2 finished with value: 493.28622084295034 and parameters: {'learning_rate': 0.050133383885928647, 'num_leaves': 25, 'subsample': 0.9913365883671131, 'colsample_bytree': 0.7157220430684382, 'lambda_l1': 3.7504747732709007, 'lambda_l2': 6.4406689894

--- Search Complete ---
Best Validation RMSE (for target_6m_ahead): 468.54
Best Parameters Found:
{'learning_rate': 0.08056583009280904, 'num_leaves': 21, 'subsample': 0.9525650463826345, 'colsample_bytree': 0.850318972819252, 'lambda_l1': 6.918875094886156, 'lambda_l2': 0.3219122718364551, 'min_child_samples': 37}


In [ ]:
# 1. Get the best parameters from the study
# Best Parameters Found:
# {'learning_rate': 0.08056583009280904, 'num_leaves': 21, 'subsample': 0.9525650463826345, 'colsample_bytree': 0.850318972819252, 'lambda_l1': 6.918875094886156, 'lambda_l2': 0.3219122718364551, 'min_child_samples': 37}#

best_params = study.best_params

# 2. Set up final parameters
final_params = {
    'objective': 'regression_l1',
    'metric': 'rmse',
    'seed': 42,
    'n_jobs': -1,
    'verbose': -1,
    'n_estimators': 2000, # Use a high number, early stopping will handle it
    'feature_pre_filter': False, # Important!
    **best_params  # Unpacks all the best params (lr, num_leaves, etc.)
}

# 3. Train all 12 models in a loop
print("--- Training All 12 Models ---")
model_pipeline = {} # This will hold our 12 models
HORIZON = 12

for i in range(1, HORIZON + 1):
    target_name = f'target_{i}m_ahead'
    print(f"Training model for: {target_name}")
    
    # Get the specific 'y' for this model
    y_train_model = train[target_name]
    y_val_model = val[target_name]
    
    # Create the datasets
    train_data_model = lgb.Dataset(X_train, label=y_train_model, categorical_feature=static_features)
    val_data_model = lgb.Dataset(X_val, label=y_val_model, reference=train_data_model)
    
    # Train the model
    model = lgb.train(
        final_params,
        train_data_model,
        valid_sets=[val_data_model],
        valid_names=['validation'],
        callbacks=[lgb.early_stopping(50, verbose=False)] # No logging, just stop
    )
    
    # Store the trained model
    model_pipeline[i] = model

print("\n--- Training Complete: 12 models stored in 'model_pipeline' ---")

--- Training All 12 Models ---
Training model for: target_1m_ahead
Training model for: target_2m_ahead
Training model for: target_3m_ahead
Training model for: target_4m_ahead
Training model for: target_5m_ahead
Training model for: target_6m_ahead
Training model for: target_7m_ahead
Training model for: target_8m_ahead
Training model for: target_9m_ahead
Training model for: target_10m_ahead
Training model for: target_11m_ahead
Training model for: target_12m_ahead

--- Training Complete: 12 models stored in 'model_pipeline' ---


In [13]:
print("--- Starting Direct 12-Month Forecast ---")

# 1. Get the last known row of features
# This is the row for Sep 2024, which has all the real,
# historical lags we need to make a forecast.
# 'data' is the full DataFrame from our (now fixed) Cell 2.
latest_features_df = data.sort_values(by=TIME_COLUMN).tail(1)[features]

print(f"Using features from: {data.sort_values(by=TIME_COLUMN).tail(1)[TIME_COLUMN].dt.strftime('%Y-%m').values[0]}")

# 2. Create a DataFrame to hold the forecast
# Get the dates for our 12 forecast months
last_train_date = data[TIME_COLUMN].max()
future_dates = [last_train_date + pd.DateOffset(months=i) for i in range(1, 13)]
forecast_df = pd.DataFrame({TIME_COLUMN: future_dates})

# 3. Loop 12 times and predict
# 'model_pipeline' holds our 12 models from Cell 3.B
for i in range(1, 13):
    model_name = f'month_{i}_pred'
    target_name = f'target_{i}m_ahead'
    
    # Get the model trained for this horizon
    model = model_pipeline[i]
    
    # Predict using only the single row of real history
    prediction = model.predict(latest_features_df)
    
    # Store the prediction
    forecast_df[model_name] = prediction[0]

print("--- Direct Forecast Complete ---")

# 4. Format the output for scoring
# We need to "unpivot" the table from wide to long
forecast_df_long = forecast_df.melt(
    id_vars=[TIME_COLUMN], 
    var_name='model', 
    value_name='consumption_pred'
)

# We just want one prediction per group_id, but our models
# are "global." We need to duplicate the forecast for each group.
all_groups = data[GROUP_COLUMN].unique()
final_forecast_list = []

for group in all_groups:
    temp_df = forecast_df_long.copy()
    temp_df[GROUP_COLUMN] = group
    final_forecast_list.append(temp_df)

final_forecast_df = pd.concat(final_forecast_list)

final_forecast_df

--- Starting Direct 12-Month Forecast ---
Using features from: 2024-09
--- Direct Forecast Complete ---


,measured_at,model,consumption_pred,group_id
0,2024-10-01 00:00:00+00:00,month_1_pred,461.726564,28
1,2024-11-01 00:00:00+00:00,month_1_pred,461.726564,28
2,2024-12-01 00:00:00+00:00,month_1_pred,461.726564,28
3,2025-01-01 00:00:00+00:00,month_1_pred,461.726564,28
4,2025-02-01 00:00:00+00:00,month_1_pred,461.726564,28
...,...,...,...,...
139,2025-05-01 00:00:00+00:00,month_12_pred,359.596186,741
140,2025-06-01 00:00:00+00:00,month_12_pred,359.596186,741
141,2025-07-01 00:00:00+00:00,month_12_pred,359.596186,741
142,2025-08-01 00:00:00+00:00,month_12_pred,359.596186,741


In [14]:
# --- 1. Load the Test Set with True Values ---
TEST_TRUTH_PATH = "monthly_df_test.csv"
print(f"Loading true values from {TEST_TRUTH_PATH}...")
test_true_df = pd.read_csv(TEST_TRUTH_PATH)

# --- 2. Prepare for Merge (CRITICAL STEP) ---

# A. Convert 'measured_at' to datetime and *localize to UTC*
# This is essential to match the 'datetime64[ns, UTC]' dtype
# of our 'final_forecast_df'.
test_true_df[TIME_COLUMN] = pd.to_datetime(test_true_df[TIME_COLUMN], utc=True)

# B. Rename columns for a clean merge
# 'consumption' in our forecast is the prediction
forecast_df_renamed = final_forecast_df.rename(columns={'consumption': 'consumption_pred'})
# 'consumption' in the test file is the truth
test_true_df_renamed = test_true_df.rename(columns={'consumption': 'consumption_true'})

# --- 3. Merge Forecasts and True Values ---
# We merge on the two keys: the time and the group
keys = [TIME_COLUMN, GROUP_COLUMN]
score_df = pd.merge(
    forecast_df_renamed[keys + ['consumption_pred']],
    test_true_df_renamed[keys + ['consumption_true']],
    on=keys,
    how='inner' # Only score rows that are in both dataframes
)

# --- 4. Calculate and Print Final Scores ---
if len(score_df) == 0:
    print("\n--- FATAL ERROR: Merge failed. No matching rows found. ---")
    print("Check that 'measured_at' (including UTC) and 'group_id' values match.")
elif len(score_df) < len(final_forecast_df):
    print(f"\nWarning: Scored {len(score_df)} rows, but forecast had {len(final_forecast_df)} rows.")
    print("Some 'group_id's or 'measured_at' dates may not have been in the test file.")
else:
    print(f"\nSuccessfully merged {len(score_df)} rows for scoring.")

# Calculate metrics
mae = mean_absolute_error(score_df['consumption_true'], score_df['consumption_pred'])
rmse = np.sqrt(mean_squared_error(score_df['consumption_true'], score_df['consumption_pred']))
mape = mean_absolute_percentage_error(score_df['consumption_true'], score_df['consumption_pred'])

print("\n--- 12-Month Forecast Test Set Performance ---")
print(f"Final Test MAE:  {mae:.2f}")
print(f"Final Test RMSE: {rmse:.2f}")
print(f"Final Test MAPE: {mape:.2f}")

Loading true values from monthly_df_test.csv...

Successfully merged 16128 rows for scoring.

--- 12-Month Forecast Test Set Performance ---
Final Test MAE:  638.72
Final Test RMSE: 776.63
Final Test MAPE: 1.49
